Notebook for obtaining the fine tune model for clsoed loop system

Dependencies : 
1.) Data folder with preprocessed data
2.) Model folder for saving the models - pretrained, finetuned and tested


Importing google stuff

Extracting data from preprocessed file - 1 to 100

In [23]:
import wandb
wandb.login()

True

In [24]:
import os
from pathlib import Path
from unittest import skip
import numpy as np
import pandas as pd
import pickle
import scipy.io as sio
import random

pd.options.mode.chained_assignment = None  # default='warn

def makeBigList(lst):
    biglst = []
    for item in lst :
        biglst = biglst + [k for k in range((item-1)*10,item*10)]
    return biglst

def makeRandomValList(val,config, mode = None):
    random.seed(val)
    trial_list= [ (val-1)*10 + k for k in range(0,10)]
    random.shuffle(trial_list)
    if mode == 'pre':
        return trial_list[:4],trial_list[4:6],trial_list[6:]
    elif mode == 'fine':
        return trial_list[:config['train_size']], trial_list[config['train_size']:config['train_size']+config['val_size']],trial_list[config['train_size']+config['val_size']:] 
    
# have make finetunepretraindata which takes in train subject list,val sub
# if pretrain return train data if label == train, val data if label == val

def data_gen(train, val, test, config, mode = None):
    
    random.seed(test)
    pre_path = Path("./data/MIData_filtord2_freqlimits['1_100Hz']_ws500.pkl")
    #pre_path = "/content/drive/MyDrive/myMUPS/MIData_filtord2_freqlimits['1_100Hz']_ws500.pkl"
    a_file = open(pre_path, "rb")
    data_dict = pickle.load(a_file)
    X = data_dict['data']
    y = data_dict['labels']
    
    val_list = makeRandomValList(val, config, mode = 'pre')
    #print(val_list)

    data = []
    label = []
    if mode == 'train':
        for df in X:
            if df in makeBigList(train)+val_list[0]:
                for segment in range(len(X[df])):
                    # only legs and relax
                    if y[df][segment] == 0 or y[df][segment] == 3: # label for relax is 0, and legs is 3
                        data.append(X[df][segment])
                        if y[df][segment] == 3:
                            label.append(y[df][segment]-2)
                        else :
                            label.append(y[df][segment])
                            
    elif mode == 'val':
        for df in X:
            if df in val_list[1]:
                for segment in range(len(X[df])):
                  # only legs and relax
                  if y[df][segment] == 0 or y[df][segment] == 3: # label for relax is 0, and legs is 3
                        data.append(X[df][segment])
                        if y[df][segment] == 3:
                            label.append(y[df][segment]-2)
                        else :
                            label.append(y[df][segment])
                    
    elif mode == 'test':
        for df in X:
            if df in val_list[2]:
                for segment in range(len(X[df])):
                    # only legs and relax
                    if y[df][segment] == 0 or y[df][segment] == 3: # label for relax is 0, and legs is 3
                        data.append(X[df][segment])
                        if y[df][segment] == 3:
                            label.append(y[df][segment]-2)
                        else :
                            label.append(y[df][segment])                    
    
    
    data = np.stack(data)
    label = np.stack(label)
    
    return data, label

Loading MUPS pt data according to Pretraining for fine tuning

In [25]:
import numpy as np
# import self defined functions 
from torch.utils.data import Dataset
import scipy.io as sio

class LoadmyMUPSptdata(Dataset):

    def __init__(self, setname, train, val, test, config, train_aug=False):
        self.config = config

        if setname == 'train':
            print('preparing training data')
            # generating training data
            train_dataset = data_gen(train, val, test, config, mode = setname)
            train_X = train_dataset[0] # data
            train_y = train_dataset[1] # label

            # shuffling training data
            idx = list(range(len(train_y)))
            np.random.shuffle(idx)
            train_X = train_X[idx]
            train_y = train_y[idx]

            # doing load MUPS data operations
            train_y = train_y.ravel()
            train_win_x = train_X[:, np.newaxis, :, :].astype('float32')
            train_win_y=train_y

            print('Train data :',train_win_x.shape)
            print('Train labels :',train_win_y.shape)           
            # outputting data for operations
            self.data=train_win_x
            self.label=train_win_y

        elif setname == 'val':
            print('Preparing validation data')
            # generating validation data
            val_dataset = data_gen(train, val, test, config, mode = setname)
            val_X = val_dataset[0] # data
            val_y = val_dataset[1] # label

            # shuffling training data
            idx = list(range(len(val_y)))
            np.random.shuffle(idx)
            val_X = val_X[idx]
            val_y = val_y[idx]

            # doing load MUPS data operations
            val_y = val_y.ravel()
            val_win_x = val_X[:, np.newaxis, :, :].astype('float32')
            val_win_y= val_y

            # outputting data for operations
            self.data = val_win_x
            self.label=val_win_y
            print('Val data :',val_win_x.shape)
            print('Val labels :',val_win_y.shape)
            self.X_val=val_win_x
            self.y_val=val_win_y

        elif setname == 'test':
            print('Preparing test data')
            # generating test data
            test_dataset = data_gen(train, val, test, config, mode = setname)
            test_X = test_dataset[0] # data
            test_y = test_dataset[1] # label

            # shuffling training data
            idx = list(range(len(test_y)))
            np.random.shuffle(idx)
            test_X = test_X[idx]
            test_y = test_y[idx]

            # doing load MUPS data operations
            test_y = test_y.ravel()
            test_win_x = test_X[:, np.newaxis, :, :].astype('float32')
            test_win_y= test_y
            
            print('Test data :',test_win_x.shape)
            print('Test labels :',test_win_y.shape)
            # outputting data for operations
            self.data=test_win_x
            self.label=test_win_y

        self.num_class=self.config['way']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        data, label=self.data[i], self.label[i]
        return data, label

Samplers.py

In [26]:
""" Sampler for dataloader. """
import torch
import numpy as np

class CategoriesSampler():
    """The class to generate episodic data"""
    def __init__(self, label, n_batch, n_cls, n_per):
        self.n_batch = n_batch
        self.n_cls = n_cls
        self.n_per = n_per

        label = np.array(label)
        self.m_ind = []
        for i in range(n_cls):
            ind = np.argwhere(label == i).reshape(-1)
            ind = torch.from_numpy(ind)
            self.m_ind.append(ind)

    def __len__(self):
        return self.n_batch
    def __iter__(self):
        for i_batch in range(self.n_batch):
            batch = []
            for c in range(self.n_cls):
                l = self.m_ind[c]
                pos = torch.randperm(len(l))[:self.n_per]
                batch.append(l[pos])
            batch = torch.stack(batch).t().reshape(-1)
            yield batch

Feature Extractor py file

In [27]:
""" Feature Extractor EEGNet model definition"""

#receptive_field = 64
#filter_sizing = 8
#dropout = 0.1
#D = 2  

import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class FeatureExtractor(nn.Module):

    def __init__(self, config, mtl=True): #add parameters here wandb and optimization
        super(FeatureExtractor, self).__init__()
        self.config = config
        receptive_field = 64
        filter_sizing = 8
        dropout = 0.1
        D = 2
        sample_duration = 500
        channel_amount = 8
        if self.config['clstype'] == 'legs':
            num_classes = 2
        elif self.config['clstype'] == 'multiclass':
            num_classes = 4  
        self.temporal=nn.Sequential(
            nn.Conv2d(1,filter_sizing,kernel_size=[1,receptive_field],stride=1, bias=False,\
                padding='same'), 
            nn.BatchNorm2d(filter_sizing),
        )
        self.spatial=nn.Sequential(
            nn.Conv2d(filter_sizing,filter_sizing*D,kernel_size=[channel_amount,1],bias=False,\
                groups=filter_sizing),
            nn.BatchNorm2d(filter_sizing*D),
            nn.ReLU(True),
        )
        self.seperable=nn.Sequential(
            nn.Conv2d(filter_sizing*D,filter_sizing*D,kernel_size=[1,16],\
                padding='same',groups=filter_sizing*D, bias=False),
            nn.Conv2d(filter_sizing*D,filter_sizing*D,kernel_size=[1,1], padding='same',groups=1, bias=False),
            nn.BatchNorm2d(filter_sizing*D),
            nn.ReLU(True),
        )
        self.avgpool1 = nn.AvgPool2d([1, 4], stride=[1, 4], padding=0)   
        self.avgpool2 = nn.AvgPool2d([1, 8], stride=[1, 8], padding=0)
        self.dropout = nn.Dropout(dropout)
        self.view = nn.Sequential(Flatten())

        endsize = filter_sizing*D*15

    def forward(self,x):
        out = self.temporal(x)
        out = self.spatial(out)
        out = self.avgpool1(out)
        out = self.dropout(out)
        out = self.seperable(out)
        out = self.avgpool2(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        #output shape of out = [batch_size, 208] #240 in new model
        #using out instead of prediction to adapt it to MUPS code
        #prediction = self.fc2(out)
        #output shape of prediction = [batch_size,4]
        return out

Meta transfer Learner Base model

In [28]:
""" Meta Learner """
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
class BaseLearner(nn.Module):
    """The class for inner loop."""
    def __init__(self, config, z_dim):
        super().__init__()
        self.config = config
        self.z_dim = z_dim #208 # 240 for new model
        self.vars = nn.ParameterList()
        self.fc2_w = nn.Parameter(torch.ones([self.config['way'], self.z_dim]))
        # config['way'] --> Way number, how many classes in a task
        torch.nn.init.kaiming_normal_(self.fc2_w)
        self.vars.append(self.fc2_w)
        self.fc2_b = nn.Parameter(torch.zeros(self.config['way']))
        self.vars.append(self.fc2_b)

    def forward(self, input_x, the_vars=None):
        if the_vars is None:
            the_vars = self.vars
        fc2_w = the_vars[0]
        fc2_b = the_vars[1]
        net = F.softmax(F.linear(input_x, fc2_w, fc2_b), dim=1)
        return net

    def parameters(self):
        return self.vars

class MtlLearner(nn.Module):
    """The class for outer loop."""
    def __init__(self, config, mode='meta'):
        super().__init__()
        self.config = config
        self.mode = mode
        self.update_lr = config['base_lr']
        self.update_step = config['update_step']
        #z_dim = filter_sizing*D*13 ~ 8*2*13 = 208 # 240 for new model
        z_dim = 240
        if self.config['clstype'] == 'legs':
            num_cls = 2
        elif self.config['clstype'] == 'multiclass':
            num_cls = 4  
        self.base_learner = BaseLearner(config, z_dim)

        if self.mode == 'meta':
            self.encoder = FeatureExtractor(self.config)  
        else:
            self.encoder = FeatureExtractor(self.config, mtl=False)  
            self.pre_fc = nn.Sequential(nn.Linear(z_dim, num_cls))

    def forward(self, inp):
        if self.mode=='pre' or self.mode=='origval':
            return self.pretrain_forward(inp)
        elif self.mode=='meta':
            data_shot, label_shot, data_query, type = inp
            return self.meta_forward(data_shot, label_shot, data_query, type)
        elif self.mode=='preval':
            data_shot, label_shot, data_query = inp
            return self.preval_forward(data_shot, label_shot, data_query)
        else:
            raise ValueError('Please set the correct mode.')

    def pretrain_forward(self, inp):
        return F.softmax(self.pre_fc(self.encoder(inp)), dim=1)    

    def preval_forward(self, data_shot, label_shot, data_query):
        embedding_query = self.encoder(data_query)
        embedding_shot = self.encoder(data_shot)
        logits = self.base_learner(embedding_shot)
        loss = F.cross_entropy(logits, label_shot)
        grad = torch.autograd.grad(loss, self.base_learner.parameters())
        fast_weights = list(map(lambda p: p[1] - 0.005 * p[0], zip(grad, self.base_learner.parameters())))
        logits_q = self.base_learner(embedding_query, fast_weights)

        for _ in range(2):
            logits = self.base_learner(embedding_shot, fast_weights)
            loss = F.cross_entropy(logits, label_shot)
            grad = torch.autograd.grad(loss, fast_weights)
            fast_weights = list(map(lambda p: p[1] - 0.005 * p[0], zip(grad, fast_weights)))
            logits_q = self.base_learner(embedding_query, fast_weights)
        return logits_q

    def meta_forward(self, data_shot, label_shot, data_query, type = None):
        embedding_shot = self.encoder(data_shot)
        embedding_query = self.encoder(data_query)
        params=self.base_learner.parameters()
        optimizer=optim.Adam(params)
        logits = self.base_learner(embedding_shot)
        loss = F.cross_entropy(logits, label_shot)
        loss.backward(retain_graph=True)
        optimizer.step()
        logits_q = self.base_learner(embedding_query)

        for _ in range(10):
            optimizer.zero_grad()
            logits = self.base_learner(embedding_shot)
            loss = F.cross_entropy(logits, label_shot)
            loss.backward(retain_graph=True)
            optimizer.step()
            logits_q = self.base_learner(embedding_query)
        if type == 'test':
            return logits_q, self.base_learner.state_dict()
        else :
            return logits_q

Misc files

In [29]:
""" Tools for GPU. """
import os
import torch

def set_gpu(cuda_device):
    os.environ['CUDA_VISIBLE_DEVICES'] = cuda_device
    print('Using gpu:', cuda_device)

In [30]:
""" Additional utility functions. """
import os
import time
import pprint
import torch
import numpy as np
import torch.nn.functional as F

def subName(test):
    if test < 10:
        sub = 'X0'+str(test)
    else :
        sub = 'X'+str(test)
    return sub

def ensure_path(path):
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)

class Averager():
    def __init__(self):
        self.n = 0
        self.v = 0

    def add(self, x):
        self.v = (self.v * self.n + x) / (self.n + 1)
        self.n += 1

    def item(self):
        return self.v

def count_acc(logits, label):
    pred = F.softmax(logits, dim=1).argmax(dim=1)
    if torch.cuda.is_available():
        return (pred == label).type(torch.cuda.FloatTensor).mean().item()
    return (pred == label).type(torch.FloatTensor).mean().item()

class Timer():
    def __init__(self):
        self.o = time.time()

    def measure(self, p=1):
        x = (time.time() - self.o) / p
        x = int(x)
        if x >= 3600:
            return '{:.1f}h'.format(x / 3600)
        if x >= 60:
            return '{}m'.format(round(x / 60))
        return '{}s'.format(x)

def compute_confidence_interval(data):
    a = 1.0 * np.array(data)
    m = np.mean(a)
    std = np.std(a)
    pm = 1.96 * (std / np.sqrt(len(a)))
    return m, pm
    
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=1e-4):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

Pre Trainer

In [31]:
""" Trainer for pretrain phase. """

import os.path as osp
import os
import tqdm
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable

  
def pre_train(config=None):
    """The function for the pre-train phase."""

    # Set the pretrain log
    trlog = {}
    #trlog['config'] = vars(self.config)
    trlog['train_loss'] = []
    trlog['val_loss'] = []
    trlog['train_acc'] = []
    trlog['val_acc'] = []
    trlog['max_acc'] = 0.0
    trlog['max_acc_epoch'] = 0

    # Set the timer
    timer = Timer()
    # Set global count to zero
    global_count = 0

    # early stopping to avoid overfitting of data - 7 epochs in question
    early_stopping = EarlyStopping()
#    LoadmyMUPSptdata --> def __init__(self, setname, train, val, test, config, train_aug=False):
    # Load pretrain set
    print("Preparing dataset loader")
    trainset = LoadmyMUPSptdata('train', config['train'], config['val'][0], config['test'][0], config, train_aug=False)
    train_loader = DataLoader(dataset=trainset, batch_size=config['pre_batch_size'], shuffle=True, num_workers=4, pin_memory=True)

    # Load meta-val set
    valset = LoadmyMUPSptdata('val', config['train'], config['val'][0], config['test'][0], config)
    val_sampler = CategoriesSampler(valset.label, 20, config['way'], config['shot'] + config['val_query'])
    val_loader = DataLoader(dataset=valset, batch_sampler=val_sampler, num_workers=4, pin_memory=True)

    # Set pretrain class number 
    #num_class_pretrain = trainset.num_class
    
    # Build pretrain model
    model = MtlLearner(config, mode='pre')
    #model=model.float()
    # Set optimizer
    params=list(model.encoder.parameters())+list(model.pre_fc.parameters()) 
    optimizer=optim.Adam(params)
    
    # Set model to GPU
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        model = model.cuda()

    # wandb.watch(model, log_freq=100)
    # Start pretrain
    for epoch in range(1, config['pre_max_epoch'] + 1): #30
        # Set the model to train mode

        print('Epoch {}'.format(epoch))
        model.train()
        model.mode = 'pre'
        # Set averager classes to record training losses and accuracies
        train_loss_averager = Averager()
        train_acc_averager = Averager()
            
        # Using tqdm to read samples from train loader
        

        tqdm_gen = tqdm.tqdm(train_loader)
        #for i, batch in enumerate(train_loader):
        for i, batch in enumerate(val_loader, 1)
        #for i, batch in enumerate(tqdm_gen, 1):
            # Update global count number 
            global_count = global_count + 1
            if torch.cuda.is_available():
                data, _ = [_.cuda() for _ in batch]
            else:
                data = batch[0]
            label = batch[1]
            if torch.cuda.is_available():
                label = label.type(torch.cuda.LongTensor)
            else:
                label = label.type(torch.LongTensor)
            logits = model(data) # modified
            loss = F.cross_entropy(logits, label)
            # Calculate train accuracy
            acc = count_acc(logits, label)
            # Print loss and accuracy for this step
            train_loss_averager.add(loss.item())
            train_acc_averager.add(acc)
            # Loss backwards and optimizer updates
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Update the averagers
        train_loss_averager = train_loss_averager.item()
        train_acc_averager = train_acc_averager.item()
        
        # start the original evaluation
        model.eval()
        model.mode='origval'
        #print('valset.y_val',valset.y_val)
        _, valid_results=val_orig(model,valset.X_val,valset.y_val)
        print ('validation accuracy ', valid_results[0])
        
        # Start validation for this epoch, set model to eval mode
        model.eval()
        model.mode = 'preval'

        # Set averager classes to record validation losses and accuracies
        val_loss_averager = Averager()
        val_acc_averager = Averager()

        # Generate the labels for test 
        label = torch.arange(config['way']).repeat(config['val_query'])
        if torch.cuda.is_available():
            label = label.type(torch.cuda.LongTensor)
        else:
            label = label.type(torch.LongTensor)
        label_shot = torch.arange(config['way']).repeat(config['shot'])
        if torch.cuda.is_available():
            label_shot = label_shot.type(torch.cuda.LongTensor)
        else:
            label_shot = label_shot.type(torch.LongTensor)
          
        # Run meta-validation
        for i, batch in enumerate(val_loader, 1):
            if torch.cuda.is_available():
                data, _ = [_.cuda() for _ in batch]
            else:
                data = batch[0]
            #data=data.float()
            p = config['shot'] * config['way']
            data_shot, data_query = data[:p], data[p:]
            logits = model((data_shot, label_shot, data_query)) # modified
            loss = F.cross_entropy(logits, label)
            acc = count_acc(logits, label)
            val_loss_averager.add(loss.item())
            val_acc_averager.add(acc)

        # Update validation averagers
        val_loss_averager = val_loss_averager.item()
        val_acc_averager = val_acc_averager.item()       

        # Update best saved model
        if val_acc_averager > trlog['max_acc']:
            trlog['max_acc'] = val_acc_averager
            trlog['max_acc_epoch'] = epoch
            save_model(model,config['test'][0],'pretrain',config['chosen_val'],mode='pt')

        # Update the logs
        trlog['train_loss'].append(train_loss_averager)
        trlog['train_acc'].append(train_acc_averager)
        trlog['val_loss'].append(val_loss_averager)
        trlog['val_acc'].append(val_acc_averager)

        if epoch % 5 == 0: # change to 5
            print('Running Time: {}, Estimated Time: {}'.format(timer.measure(), timer.measure(epoch / config['pre_max_epoch'])))

        early_stopping(val_loss_averager)
        if early_stopping.early_stop:
            break

    return epoch, train_loss_averager, train_acc_averager, val_loss_averager, val_acc_averager, trlog['max_acc']

def val_orig(model,X_val, y_val):
    predicted_loss=[]
    inputs = torch.from_numpy(X_val)
    labels = torch.FloatTensor(y_val*1.0)
    inputs, labels = Variable(inputs), Variable(labels)
    
    results = []
    predicted = []
            
    model.eval()
    model.mode = 'origval'
    

    if torch.cuda.is_available():
        inputs= inputs.type(torch.cuda.FloatTensor)
    else:
        inputs = inputs.type(torch.FloatTensor)

    predicted=model(inputs)
    predicted= predicted.data.cpu().numpy()

    Y=labels.data.numpy()
    predicted=np.argmax(predicted, axis=1)        
    for param in ["acc", "auc", "recall", "precision","fmeasure"]:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted), average='micro'))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted), average='micro')
            recall = recall_score(Y, np.round(predicted), average='micro')
            results.append(2*precision*recall/ (precision+recall))
    
    return predicted, results

Recreating the best model

In [32]:
import os
import os.path as osp

def save_model(model,sub,method,str_value,mode):
        """The function to save checkpoints.
        Args:
          model : ML model
          sub : subject
          str_value : validation set values
          mode : either pt for pretraining and mt for metatraining
          method : pretrained or finetuned
        """  
        sub = subName(sub)
        save_path = Path("./closedloop/models/")
#         save_path = f'/content/drive/MyDrive/myMUPS/closedloop/models'
        if not osp.exists(save_path):
            os.mkdir(save_path)
        save_path = osp.join(save_path, sub)
        if not osp.exists(save_path):
            os.mkdir(save_path)
        save_path = osp.join(save_path, method)
        if not osp.exists(save_path):
            os.mkdir(save_path) 
         
        torch.save(dict(params=model.encoder.state_dict()), osp.join(save_path, f'{str(sub)}_max_acc_legs_{mode}_{str_value}' + '.pth'))

Data loading of test subject into train val and test

In [33]:
import os
from pathlib import Path
from unittest import skip
import numpy as np
import pandas as pd
import pickle
import scipy.io as sio
import random

pd.options.mode.chained_assignment = None  # default='warn  
    
# have make finetunepretraindata which takes in train subject list,val sub
# if pretrain return train data if label == train, val data if label == val

def data_gen_test(test, config, mode = None):
    
    sub = subName(test)
    print('Test subject:', sub)
    pre_path = Path(f"./data/preprocess/colab/{sub}_wet_filtord2_freqlimits['1_100Hz']_ws500.pkl")
    #pre_path = f"/content/drive/MyDrive/myMUPS/closedloop/data/"
    # X0_wet_filtord2_freqlimits['1_100Hz']_ws500
    a_file = open(pre_path, "rb")
    data_dict = pickle.load(a_file)
    X = data_dict['data']
    y = data_dict['labels']

    trials = [k for k in range(0,5)]
    val = config['test_val']
    trials = list(set(trials)-set([val]))
    val_list = trials[:3], [val], [trials[3]]
    #print(val_list)
    
    data = []
    label = []
    if mode == 'train':
        for df in X:
            if df in val_list[0]:
                for segment in range(len(X[df])):
                    data.append(X[df][segment])
                    label.append(y[df][segment])
                            
    elif mode == 'val':
        for df in X:
            if df in val_list[1]:
                for segment in range(len(X[df])):
                    data.append(X[df][segment])
                    label.append(y[df][segment])
                    
    elif mode == 'test':
        for df in X:
            if df in val_list[2]:
                for segment in range(len(X[df])):
                    data.append(X[df][segment])
                    label.append(y[df][segment])                   
    
    
    data = np.stack(data)
    label = np.stack(label)
    
    return data, label

Making changes to LoadmyMUPSData to make it give us test data 

In [34]:
import numpy as np
# import self defined functions 
from torch.utils.data import Dataset
import scipy.io as sio

class LoadmyMUPStestdata(Dataset):

    def __init__(self, setname, test, config):
        self.config = config

        if setname == 'train':
            print('preparing training data')
            # generating training data
            train_dataset = data_gen_test(test, config, mode = setname)
            train_X = train_dataset[0] # data
            train_y = train_dataset[1] # label

            # shuffling training data
            idx = list(range(len(train_y)))
            np.random.shuffle(idx)
            train_X = train_X[idx]
            train_y = train_y[idx]

            # doing load MUPS data operations
            train_y = train_y.ravel()
            train_win_x = train_X[:, np.newaxis, :, :].astype('float32')
            train_win_y=train_y

            print('Train data :',train_win_x.shape)
            print('Train labels :',train_win_y.shape)           
            # outputting data for operations
            self.data=train_win_x
            self.label=train_win_y

        elif setname == 'val':
            print('Preparing validation data')
            # generating validation data
            val_dataset = data_gen_test(test, config, mode = setname)
            val_X = val_dataset[0] # data
            val_y = val_dataset[1] # label

            # shuffling training data
            idx = list(range(len(val_y)))
            np.random.shuffle(idx)
            val_X = val_X[idx]
            val_y = val_y[idx]

            # doing load MUPS data operations
            val_y = val_y.ravel()
            val_win_x = val_X[:, np.newaxis, :, :].astype('float32')
            val_win_y= val_y

            # outputting data for operations
            self.data = val_win_x
            self.label=val_win_y
            print('Val data :',val_win_x.shape)
            print('Val labels :',val_win_y.shape)
            self.X_val=val_win_x
            self.y_val=val_win_y

        elif setname == 'test':
            print('Preparing test data')
            # generating test data
            test_dataset = data_gen_test(test, config, mode = setname)
            test_X = test_dataset[0] # data
            test_y = test_dataset[1] # label

            shape = len(test_X)//2

            # shuffling training data
            idx = list(range(len(test_y)))
            random.seed(config['seed'])
            np.random.shuffle(idx)
            test_X = test_X[idx]
            test_y = test_y[idx]


            # doing load MUPS data operations
            test_y = test_y.ravel()
            test_win_x = test_X[:, np.newaxis, :, :].astype('float32')
            test_win_y= test_y
            
            print('Test data :',test_win_x.shape)
            print('Test labels :',test_win_y.shape)
            # outputting data for operations
            self.data=test_win_x
            self.label=test_win_y

        self.num_class=self.config['way']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        data, label=self.data[i], self.label[i]
        return data, label

Final Deep Learning model for Testing

In [35]:
class MindReader(nn.Module):

    def __init__(self, mtl=True): #add parameters here wandb and optimization
        super(MindReader, self).__init__()
        receptive_field = 64
        filter_sizing = 8
        dropout = 0.1
        D = 2
        channel_amount = 8
        num_classes = 2
        self.temporal=nn.Sequential(
            nn.Conv2d(1,filter_sizing,kernel_size=[1,receptive_field],stride=1, bias=False,\
                padding='same'), 
            nn.BatchNorm2d(filter_sizing),
        )
        self.spatial=nn.Sequential(
            nn.Conv2d(filter_sizing,filter_sizing*D,kernel_size=[channel_amount,1],bias=False,\
                groups=filter_sizing),
            nn.BatchNorm2d(filter_sizing*D),
            nn.ReLU(True),
        )
        self.seperable=nn.Sequential(
            nn.Conv2d(filter_sizing*D,filter_sizing*D,kernel_size=[1,16],\
                padding='same',groups=filter_sizing*D, bias=False),
            nn.Conv2d(filter_sizing*D,filter_sizing*D,kernel_size=[1,1], padding='same',groups=1, bias=False),
            nn.BatchNorm2d(filter_sizing*D),
            nn.ReLU(True),
        )
        self.avgpool1 = nn.AvgPool2d([1, 4], stride=[1, 4], padding=0)   
        self.avgpool2 = nn.AvgPool2d([1, 8], stride=[1, 8], padding=0)
        self.dropout = nn.Dropout(dropout)
        self.view = nn.Sequential(Flatten())

        endsize = filter_sizing*D*15
        self.fc2 = nn.Linear(endsize, num_classes)

    def forward(self,x):
        out = self.temporal(x)
        out = self.spatial(out)
        out = self.avgpool1(out)
        out = self.dropout(out)
        out = self.seperable(out)
        out = self.avgpool2(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        #output shape of out = [batch_size, 208] #240 in new model
        #using out instead of prediction to adapt it to MUPS code
        prediction = self.fc2(out)
        #output shape of prediction = [batch_size,4]
        return prediction

Code for double loop finetuning using test data as train, val and test

Meta Fine Tuner 

In [36]:
""" Trainer for meta-train phase. """
import os.path as osp
import os
import tqdm
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score
from sklearn.preprocessing import LabelBinarizer
import time


def meta_finetune(config=None, mode='finetune'):
    """The function for the meta-train phase."""
    # initializing wandb for recording
    with wandb.init(project=f"Realtime testing",config=config):
        config = wandb.config

        if mode == 'pretrain':
            # getting pretraining results and saving model
            pre_epoch, pre_train_loss, pre_train_acc, pre_val_loss, pre_val_acc, pre_max_acc = pre_train(config)

            # logging pretrain data into wandb
            wandb.log({"pre_epoch": pre_epoch,"pre_train/train_loss": pre_train_loss,
            "pre_train/train_acc": pre_train_acc,"pre_val/val_loss": pre_val_loss,
            "pre_val/val_acc": pre_val_acc, "pre_max acc":pre_max_acc})

        elif mode == 'finetune':
            # Load meta-train set
            print("Preparing dataset loader")
            trainset = LoadmyMUPStestdata('train', config['test'][0], config) 
            train_sampler = CategoriesSampler(trainset.label, config['num_batch'], config['way'], config['shot'] + config['train_query'])
            train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler, num_workers=2, pin_memory=True)

            # Load meta-val set
            valset = LoadmyMUPStestdata('val', config['test'][0], config)
            val_sampler = CategoriesSampler(valset.label, 20, config['way'], config['shot'] + config['val_query'])
            val_loader = DataLoader(dataset=valset, batch_sampler=val_sampler, num_workers=2, pin_memory=True)

            # Build meta-transfer learning model
            model = MtlLearner(config)

            # Set optimizer 
            optimizer = torch.optim.Adam([{'params': filter(lambda p: p.requires_grad, model.encoder.parameters())}, \
              {'params': model.base_learner.parameters(), 'lr': config['meta_lr2']}], lr=config['meta_lr1'])
            # Set learning rate scheduler 
            lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=config['gamma'])        

            # load pretrained model without FC classifier
            model_dict = model.state_dict()
            method = 'pretrain'
            str_value = config['chosen_val']
            sub = subName(testconfig['test'][0])
            save_path = Path(f"./closedloop/models/{sub}/{method}/")
            #save_path = f'/content/drive/MyDrive/myMUPS/closedloop/models/{sub}/{method}/'
            pretrained_dict = torch.load(osp.join(save_path, f'{sub}_max_acc_legs_pt_{str_value}' + '.pth'))['params']
            pretrained_dict = {'encoder.'+k: v for k, v in pretrained_dict.items()}
            pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
            model_dict.update(pretrained_dict)
            model.load_state_dict(model_dict)    

            # Set model to GPU
            if torch.cuda.is_available():
                torch.backends.cudnn.benchmark = True
                model = model.cuda()

            # Set the meta-train log
            trlog = {}
            max_dict = {}
            #trlog['config'] = vars(config)
            trlog['train_loss'] = []
            trlog['val_loss'] = []
            trlog['train_acc'] = []
            trlog['val_acc'] = []
            trlog['max_acc'] = 0.0
            trlog['max_acc_epoch'] = 0

            # Set the timer
            timer = Timer()
            # Set global count to zero
            global_count = 0

            # Generate the labels for train set of the episodes
            label_shot = torch.arange(config['way']).repeat(config['shot'])
            if torch.cuda.is_available():
                label_shot = label_shot.type(torch.cuda.LongTensor)
            else:
                label_shot = label_shot.type(torch.LongTensor)

            # early stopping to avoid overfitting of data - 7 epochs in question
            early_stopping = EarlyStopping()

            # Start meta-train
            # start_time = time.time()
            for epoch in range(1, config['max_epoch'] + 1):
                start_time = time.time()

                # Set the model to train mode
                model.train()
                # Set averager classes to record training losses and accuracies
                train_loss_averager = Averager()
                train_acc_averager = Averager()

                # Generate the labels for test set of the episodes during meta-train updates
                label = torch.arange(config['way']).repeat(config['train_query'])
                if torch.cuda.is_available():
                    label = label.type(torch.cuda.LongTensor)
                else:
                    label = label.type(torch.LongTensor)

                # Using tqdm to read samples from train loader
                tqdm_gen = tqdm.tqdm(train_loader)
                for i, batch in enumerate(tqdm_gen, 1):
                    # Update global count number 
                    global_count = global_count + 1
                    if torch.cuda.is_available():
                        data, _ = [_.cuda() for _ in batch]
                    else:
                        data = batch[0]
                    p = config['shot'] * config['way']
                    data_shot, data_query = data[:p], data[p:]
                    # Output logits for model
                    logits = model((data_shot, label_shot, data_query, None)) # modified
                    # Calculate meta-train loss
                    loss = F.cross_entropy(logits, label)
                    # Calculate meta-train accuracy
                    acc = count_acc(logits, label)

                    # Add loss and accuracy for the averagers
                    train_loss_averager.add(loss.item())
                    train_acc_averager.add(acc)

                    # Loss backwards and optimizer updates
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                # Update learning rate
                lr_scheduler.step()

                # Update the averagers
                train_loss_averager = train_loss_averager.item()
                train_acc_averager = train_acc_averager.item()

                print("--- %s seconds ---" % (time.time() - start_time))
                # Start validation for this epoch, set model to eval mode
                model.eval()

                # Set averager classes to record validation losses and accuracies
                val_loss_averager = Averager()
                val_acc_averager = Averager()

                # Generate the labels for test set of the episodes during meta-val for this epoch
                label = torch.arange(config['way']).repeat(config['val_query'])
                if torch.cuda.is_available():
                    label = label.type(torch.cuda.LongTensor)
                else:
                    label = label.type(torch.LongTensor)

                # Print previous information
                if epoch % 10 == 0:
                    print('Best Epoch {}, Best Val Acc={:.4f}'.format(trlog['max_acc_epoch'], trlog['max_acc']))
                # Run meta-validation
                for i, batch in enumerate(val_loader, 1):
                    if torch.cuda.is_available():
                        data, _ = [_.cuda() for _ in batch]
                    else:
                        data = batch[0]
                    p = config['shot'] * config['way']
                    data_shot, data_query = data[:p], data[p:]
                    #logits = model((data_shot, label_shot, data_query, None))
                    logits = model((data_shot, label_shot, data_query,None))
                    loss = F.cross_entropy(logits, label)
                    acc = count_acc(logits, label)

                    val_loss_averager.add(loss.item())
                    val_acc_averager.add(acc)

                # Update validation averagers
                val_loss_averager = val_loss_averager.item()
                val_acc_averager = val_acc_averager.item()

                # Print loss and accuracy for this epoch
                print('Epoch {}, Val, Loss={:.4f} Acc={:.4f}'.format(epoch, val_loss_averager, val_acc_averager))

                # Update best saved model
                if val_acc_averager > trlog['max_acc']:
                    trlog['max_acc'] = val_acc_averager
                    trlog['max_acc_epoch'] = epoch
                    save_model(model,config['test'][0],'finetune',config['chosen_val'],mode='mt')
                    max_dict = dict(params=model.encoder.state_dict())
                    A = dict(params=model.encoder.state_dict())

                # Update the logs
                trlog['train_loss'].append(train_loss_averager)
                trlog['train_acc'].append(train_acc_averager)
                trlog['val_loss'].append(val_loss_averager)
                trlog['val_acc'].append(val_acc_averager)

                wandb.log({"epoch": epoch,
                "train/train_loss": train_loss_averager,
                "train/train_acc": train_acc_averager,
                "val/val_loss": val_loss_averager,
                "val/val_acc": val_acc_averager})

                early_stopping(val_loss_averager)
                if early_stopping.early_stop:
                    break

            wandb.log({"max acc":trlog['max_acc']})
            evalmodel_dict = model.state_dict()
            max_dict = max_dict['params']
            max_dict = {'encoder.'+k: v for k, v in max_dict.items()}
            max_dict = {k: v for k, v in max_dict.items() if k in evalmodel_dict}
            evalmodel_dict.update(max_dict)

            # model eval mode
            model.load_state_dict(evalmodel_dict)
            if torch.cuda.is_available():
                model.cuda()
            model.eval()
            #model.mode = 'test'

            # Load meta-test set
            test_set = LoadmyMUPStestdata('test',config['test'][0], config)
            split = 40
            hdata = test_set.data[split:]
            ldata = test_set.label[split:]
            test_set.data = test_set.data[:split]
            test_set.label = test_set.label[:split] 
            test_sampler = CategoriesSampler(test_set.label, 20, config['way'], config['shot'] + config['val_query'])
            test_loader = DataLoader(test_set, batch_sampler=test_sampler, num_workers=2, pin_memory=True)

            print('Pre test length:',test_set.data.shape, test_set.label.shape )
            print('Actual test length :', hdata.shape, ldata.shape)
            # Set test accuracy recorder
            test_acc_record = np.zeros((20,))
            test_f1_record = np.zeros((20,))
            test_auc_record=np.zeros((20,))

            # Set accuracy averager
            ave_acc = Averager()

            # Generate labels
            label = torch.arange(config['way']).repeat(config['val_query'])
            if torch.cuda.is_available():
                label = label.type(torch.cuda.LongTensor)
            else:
                label = label.type(torch.LongTensor)
            label_shot = torch.arange(config['way']).repeat(config['shot'])
            if torch.cuda.is_available():
                label_shot = label_shot.type(torch.cuda.LongTensor)
            else:
                label_shot = label_shot.type(torch.LongTensor)

            Y=label.data.cpu().numpy()            
            # Start meta-test

            for i, batch in enumerate(test_loader, 1):
                if torch.cuda.is_available():
                    data, _ = [_.cuda() for _ in batch]
                else:
                    data = batch[0]
                k = config['way'] * config['shot'] # 2*20
                data_shot, data_query = data[:k], data[k:]
                logits, D = model((data_shot, label_shot, data_query, 'test'))
                acc = count_acc(logits, label)
                logits = logits.data.cpu().numpy()
                predicted=np.argmax(logits, axis=1)
                f1 = f1_score(Y,predicted, average='macro') 
                auc=multiclass_roc_auc_score(Y, predicted)
                ave_acc.add(acc)
                test_acc_record[i-1] = acc
                test_f1_record[i-1] =f1
                test_auc_record[i-1]=auc

                if i % 100 == 0:
                    print('batch {}: {:.2f}({:.2f})'.format(i, ave_acc.item() * 100, acc * 100))


            # Calculate the confidence interval, update the logs
            m, pm = compute_confidence_interval(test_acc_record)
            f1_m, f1_pm=compute_confidence_interval(test_f1_record)
            auc_m, auc_pm=compute_confidence_interval(test_auc_record)

            #print('Val Best Epoch {}, Acc {:.4f}, Test Acc {:.4f}'.format(trlog['max_acc_epoch'], trlog['max_acc'], ave_acc.item()))
            # print('Test Acc {:.4f} + {:.4f}'.format(m, pm))
            # print('Test f1 {:.4f} + {:.4f}'.format(f1_m, f1_pm))
            # print('Test auc {:.4f} + {:.4f}'.format(auc_m, auc_pm))
            wandb.log({"test/test_acc": m,
                    "test/test_f1": f1_m,
                    "test/test_auc": auc_m})

            B = A['params']
            B['fc2.weight'] = D['fc2_w']
            B['fc2.bias'] = D['fc2_b']

            print('Pre Test Accuracy :',m)
            net = MindReader()
            net.load_state_dict(B)
            #net = net.float()
            net.eval()
            ftest_X = hdata
            ftest_Y = ldata

            ftest_X = torch.from_numpy(ftest_X)
            preds = []

            for sample in ftest_X:
                sample = sample[np.newaxis, :, :, :]
                logits = net(sample)
                logits = logits.data.cpu().numpy()
                predicted=np.argmax(logits, axis=1)
                prediction = predicted[-1]
                preds.append(prediction)

            preds = np.array(preds)
            print('Actual Test Accuracy :',accuracy_score(ftest_Y, preds))
            wandb.log({"Actual test_acc": accuracy_score(ftest_Y, preds)})
            return B, m, accuracy_score(ftest_Y, preds)

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test) 
    y_pred = lb.transform(y_pred) 
    return roc_auc_score(y_test, y_pred, average=average)

Master control : main block of code to run the script

Run this for pretraining and finetuning the models

Has 2 modes - pretrain and fine tune, pre train first to save time, then run with mode fine tune

In [37]:
import wandb
import torch

# Set the GPU id
if torch.cuda.is_available(): 
    print('Using device :', torch.cuda.get_device_name(torch.cuda.current_device()))

seed = 42
# Set manual seed for PyTorch
if seed == 0:
    print('Using random seed.')
    torch.backends.cudnn.benchmark = True
else:
    print('Using manual seed:', seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# for pretraining alone
subj_list = [k for k in range(1,10)]
test_subject = 2 #update subject here
val_list = [4,5,6,7] # have your list of val subjects here
dict_list = []
for val in val_list:
#val = 4
    #mode = 'finetune' # run pretrain first
    mode = 'pretrain' # then run with mode finetune

    if test_subject in range(1,10):
        print('Old subject')
        train_list = list(set(subj_list) - set([test_subject]))
        #val = random.choice(train_list)
        train_list = list(set(train_list)- set([val]))
    else :
        print('New subject')
        #val = random.choice(subj_list)
        train_list = list(set(subj_list) - set([val]))  

    print(f"Running model for test subject {test_subject} with validation {val}")
    if mode == 'pretrain':
        config = {'clstype' :'legs','max_epoch': 2,'pre_max_epoch': 5,'seed':42,'chosen_val':str(val),#'test_val': x, #'train_size': 4,'val_size': 4
            "pre_lr": 0.05,"pre_gamma": 0.3,"update_step":20,"pre_step_size":5,"pre_batch_size":18,"gamma":0.3,
            "meta_lr1":0.0001,"meta_lr2":0.001,"num_batch":12,"step_size":2,'shot':5,'way':2,'train_query':1,'val_query':1,
            'base_lr':0.00001,"train":train_list,"val":[val],"test":[test_subject],'embed_size':200}
        meta_finetune(config, mode=mode)
    else :
        for x in range(0,5):
            config = {'clstype' :'legs','max_epoch': 2,'pre_max_epoch': 5,'seed':42,'chosen_val':str(val),'test_val': x, #'train_size': 4,'val_size': 4
                    "pre_lr": 0.05,"pre_gamma": 0.3,"update_step":20,"pre_step_size":5,"pre_batch_size":18,"gamma":0.3,
                    "meta_lr1":0.0001,"meta_lr2":0.001,"num_batch":12,"step_size":2,'shot':5,'way':2,'train_query':1,'val_query':1,
                    'base_lr':0.00001,"train":train_list,"val":[val],"test":[test_subject],'embed_size':200}
            print('For val trial :', x)
            B, pretest_acc , test_acc = meta_finetune(config, mode=mode)
            dict_list.append(((B,pretest_acc),test_acc))

Using device : NVIDIA GeForce MX150
Using manual seed: 42
Old subject
Running model for test subject 2 with validation 4


Preparing dataset loader
preparing training data
Train data : (3841, 1, 8, 500)
Train labels : (3841,)
Preparing validation data
Val data : (103, 1, 8, 500)
Val labels : (103,)
Epoch 1



  0%|                                                                                          | 0/214 [00:00<?, ?it/s]

KeyError: '__getstate__'

In [ ]:
def GetBestDict(lst):
    max_acc = 0
    max_dict = {}
    for pair in lst:
    print('Pre Test Accuracy:',pair[0][1])
    print('Actual Test Accuracy:',pair[1])
    if pair[1] > max_acc :
        max_acc = pair[1]
        max_dict = pair[0][0]
    print('Returning max accuracy dict', max_acc)
    return max_dict, max_acc

In [ ]:
B, max = GetBestDict(dict_list)
# take actual test accuracy into account while selecting model

Pre Test Accuracy: 0.55
Actual Test Accuracy: 0.42105263157894735
Pre Test Accuracy: 0.65
Actual Test Accuracy: 0.47368421052631576
Pre Test Accuracy: 0.575
Actual Test Accuracy: 0.34210526315789475
Pre Test Accuracy: 0.5
Actual Test Accuracy: 0.42105263157894735
Pre Test Accuracy: 0.875
Actual Test Accuracy: 0.7368421052631579
Pre Test Accuracy: 0.5
Actual Test Accuracy: 0.5789473684210527
Pre Test Accuracy: 0.575
Actual Test Accuracy: 0.42105263157894735
Pre Test Accuracy: 0.625
Actual Test Accuracy: 0.4473684210526316
Pre Test Accuracy: 0.55
Actual Test Accuracy: 0.5263157894736842
Pre Test Accuracy: 0.725
Actual Test Accuracy: 0.631578947368421
Pre Test Accuracy: 0.625
Actual Test Accuracy: 0.39473684210526316
Pre Test Accuracy: 0.6
Actual Test Accuracy: 0.3684210526315789
Pre Test Accuracy: 0.75
Actual Test Accuracy: 0.42105263157894735
Pre Test Accuracy: 0.675
Actual Test Accuracy: 0.5526315789473685
Pre Test Accuracy: 0.825
Actual Test Accuracy: 0.6842105263157895
Pre Test Accur

In [ ]:
def Sort_Tuple(tup):
 
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of
    # sublist lambda has been used
    tup.sort(key = lambda x: x[1], reverse = True)
    return tup

In [ ]:
bestModels = Sort_Tuple(dict_list)[:3] # getting top best 3 models here --> maybe for majority voting classifier if we get three models with >0.75 accuracy

In [ ]:
# final model is saved here
if max > 0.70 :
    sub = subName(config['test'][0])
    save_path = f'/content/drive/MyDrive/myMUPS/closedloop/models/{sub}/'
    torch.save(B, osp.join(save_path, f'{sub}_metamodel' + '.pth'))
    print('Model saved chico <3')
else :
    print('Low Accuracy Chico')

Model saved chico <3


In [ ]:
print('Trial ended successfully')

Trial ended successfully


In [15]:
# Set the pretrain log
trlog = {}
#trlog['config'] = vars(self.config)
trlog['train_loss'] = []
trlog['val_loss'] = []
trlog['train_acc'] = []
trlog['val_acc'] = []
trlog['max_acc'] = 0.0
trlog['max_acc_epoch'] = 0

# Set the timer
timer = Timer()
# Set global count to zero
global_count = 0

In [18]:
# early stopping to avoid overfitting of data - 7 epochs in question
early_stopping = EarlyStopping()
#    LoadmyMUPSptdata --> def __init__(self, setname, train, val, test, config, train_aug=False):
# Load pretrain set
print("Preparing dataset loader")
trainset = LoadmyMUPSptdata('train', config['train'], config['val'][0], config['test'][0], config, train_aug=False)
train_loader = DataLoader(dataset=trainset, batch_size=config['pre_batch_size'], shuffle=True, num_workers=2, pin_memory=True)

# Load meta-val set
valset = LoadmyMUPSptdata('val', config['train'], config['val'][0], config['test'][0], config)
val_sampler = CategoriesSampler(valset.label, 20, config['way'], config['shot'] + config['val_query'])
val_loader = DataLoader(dataset=valset, batch_sampler=val_sampler, num_workers=2, pin_memory=True)

# Set pretrain class number 
#num_class_pretrain = trainset.num_class

# Build pretrain model
model = MtlLearner(config, mode='pre')
#model=model.float()
# Set optimizer
params=list(model.encoder.parameters())+list(model.pre_fc.parameters()) 
optimizer=optim.Adam(params)

# Set model to GPU
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    model = model.cuda()

Preparing dataset loader
preparing training data
Train data : (3841, 1, 8, 500)
Train labels : (3841,)
Preparing validation data
Val data : (103, 1, 8, 500)
Val labels : (103,)


In [ ]:
for epoch in range(1, config['pre_max_epoch'] + 1): #30
    # Set the model to train mode

    print('Epoch {}'.format(epoch))
    model.train()
    model.mode = 'pre'
    # Set averager classes to record training losses and accuracies
    train_loss_averager = Averager()
    train_acc_averager = Averager()

    # Using tqdm to read samples from train loader


    tqdm_gen = tqdm.tqdm(train_loader)
    #for i, batch in enumerate(train_loader):
    for i, batch in enumerate(tqdm_gen, 1):
        # Update global count number 
        global_count = global_count + 1
        if torch.cuda.is_available():
            data, _ = [_.cuda() for _ in batch]
        else:
            data = batch[0]
        label = batch[1]
        if torch.cuda.is_available():
            label = label.type(torch.cuda.LongTensor)
        else:
            label = label.type(torch.LongTensor)
        logits = model(data) # modified
        loss = F.cross_entropy(logits, label)
        # Calculate train accuracy
        acc = count_acc(logits, label)
        # Print loss and accuracy for this step
        train_loss_averager.add(loss.item())
        train_acc_averager.add(acc)
        # Loss backwards and optimizer updates
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch 1



  0%|                                                                                          | 0/214 [00:00<?, ?it/s]

In [ ]:
""" Trainer for pretrain phase. """

import os.path as osp
import os
import tqdm
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable

  
def pre_train(config=None):
    """The function for the pre-train phase."""


    # wandb.watch(model, log_freq=100)
    # Start pretrain
    for epoch in range(1, config['pre_max_epoch'] + 1): #30
        # Set the model to train mode

        print('Epoch {}'.format(epoch))
        model.train()
        model.mode = 'pre'
        # Set averager classes to record training losses and accuracies
        train_loss_averager = Averager()
        train_acc_averager = Averager()
            
        # Using tqdm to read samples from train loader
        

        tqdm_gen = tqdm.tqdm(train_loader)
        #for i, batch in enumerate(train_loader):
        for i, batch in enumerate(tqdm_gen, 1):
            # Update global count number 
            global_count = global_count + 1
            if torch.cuda.is_available():
                data, _ = [_.cuda() for _ in batch]
            else:
                data = batch[0]
            label = batch[1]
            if torch.cuda.is_available():
                label = label.type(torch.cuda.LongTensor)
            else:
                label = label.type(torch.LongTensor)
            logits = model(data) # modified
            loss = F.cross_entropy(logits, label)
            # Calculate train accuracy
            acc = count_acc(logits, label)
            # Print loss and accuracy for this step
            train_loss_averager.add(loss.item())
            train_acc_averager.add(acc)
            # Loss backwards and optimizer updates
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Update the averagers
        train_loss_averager = train_loss_averager.item()
        train_acc_averager = train_acc_averager.item()
        
        # start the original evaluation
        model.eval()
        model.mode='origval'
        #print('valset.y_val',valset.y_val)
        _, valid_results=val_orig(model,valset.X_val,valset.y_val)
        print ('validation accuracy ', valid_results[0])
        
        # Start validation for this epoch, set model to eval mode
        model.eval()
        model.mode = 'preval'

        # Set averager classes to record validation losses and accuracies
        val_loss_averager = Averager()
        val_acc_averager = Averager()

        # Generate the labels for test 
        label = torch.arange(config['way']).repeat(config['val_query'])
        if torch.cuda.is_available():
            label = label.type(torch.cuda.LongTensor)
        else:
            label = label.type(torch.LongTensor)
        label_shot = torch.arange(config['way']).repeat(config['shot'])
        if torch.cuda.is_available():
            label_shot = label_shot.type(torch.cuda.LongTensor)
        else:
            label_shot = label_shot.type(torch.LongTensor)
          
        # Run meta-validation
        for i, batch in enumerate(val_loader, 1):
            if torch.cuda.is_available():
                data, _ = [_.cuda() for _ in batch]
            else:
                data = batch[0]
            #data=data.float()
            p = config['shot'] * config['way']
            data_shot, data_query = data[:p], data[p:]
            logits = model((data_shot, label_shot, data_query)) # modified
            loss = F.cross_entropy(logits, label)
            acc = count_acc(logits, label)
            val_loss_averager.add(loss.item())
            val_acc_averager.add(acc)

        # Update validation averagers
        val_loss_averager = val_loss_averager.item()
        val_acc_averager = val_acc_averager.item()       

        # Update best saved model
        if val_acc_averager > trlog['max_acc']:
            trlog['max_acc'] = val_acc_averager
            trlog['max_acc_epoch'] = epoch
            save_model(model,config['test'][0],'pretrain',config['chosen_val'],mode='pt')

        # Update the logs
        trlog['train_loss'].append(train_loss_averager)
        trlog['train_acc'].append(train_acc_averager)
        trlog['val_loss'].append(val_loss_averager)
        trlog['val_acc'].append(val_acc_averager)

        if epoch % 5 == 0: # change to 5
            print('Running Time: {}, Estimated Time: {}'.format(timer.measure(), timer.measure(epoch / config['pre_max_epoch'])))

        early_stopping(val_loss_averager)
        if early_stopping.early_stop:
            break

    return epoch, train_loss_averager, train_acc_averager, val_loss_averager, val_acc_averager, trlog['max_acc']

def val_orig(model,X_val, y_val):
    predicted_loss=[]
    inputs = torch.from_numpy(X_val)
    labels = torch.FloatTensor(y_val*1.0)
    inputs, labels = Variable(inputs), Variable(labels)
    
    results = []
    predicted = []
            
    model.eval()
    model.mode = 'origval'
    

    if torch.cuda.is_available():
        inputs= inputs.type(torch.cuda.FloatTensor)
    else:
        inputs = inputs.type(torch.FloatTensor)

    predicted=model(inputs)
    predicted= predicted.data.cpu().numpy()

    Y=labels.data.numpy()
    predicted=np.argmax(predicted, axis=1)        
    for param in ["acc", "auc", "recall", "precision","fmeasure"]:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted), average='micro'))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted), average='micro')
            recall = recall_score(Y, np.round(predicted), average='micro')
            results.append(2*precision*recall/ (precision+recall))
    
    return predicted, results